# AutoGluon을 이용한 California Housing 집값 예측

이 노트북에서는 California Housing 데이터셋을 사용하여 AutoGluon으로 다양한 앙상블 모델을 학습하고 튜닝합니다.
**포함된 모델:** GBM, LGBM, XGBoost, CatBoost, Random Forest, Weighted Ensemble, Stacking Ensemble

In [ ]:
# AutoGluon 설치 (필요시 주석 해제 후 실행)
# !pip install autogluon

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
# 1. 데이터 로드
housing = fetch_california_housing()
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df['MedHouseVal'] = housing.target

print("데이터 크기:", df.shape)
df.head()

In [ ]:
# 2. 학습/테스트 데이터 분리
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
print("학습 데이터:", train_data.shape)
print("테스트 데이터:", test_data.shape)

In [ ]:
# 3. AutoGluon 학습 설정
# presets='best_quality': Bagging, Stacking 등을 포함한 최적의 성능을 위한 프리셋
# hyperparameters: 특정 모델들을 강제로 포함하거나 튜닝 범위를 지정

save_path = 'ag_models_california_ensemble'  # 모델 저장 경로
label = 'MedHouseVal'                       # 타겟 컬럼
eval_metric = 'root_mean_squared_error'     # 평가지표

# 학습할 모델 지정 (요청하신 GBM, LGBM, XGBoost, CatBoost, RF 포함)
# GBM은 LightGBM을 포함하는 범주이며, XGB, CAT, RF를 명시합니다.
hyperparameters = {
    'GBM': {},      # Gradient Boosting Machine (LightGBM implementation often used)
    'CAT': {},      # CatBoost
    'XGB': {},      # XGBoost
    'RF': {},       # Random Forest
}

predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path).fit(
    train_data,
    presets='best_quality', # 고품질 앙상블 (Stacking 포함) 유도
    hyperparameters=hyperparameters,
    time_limit=3600  # 시간 제한 설정 (초 단위, 필요에 따라 조절)
)

In [ ]:
# 4. 리더보드 확인 (앙상블 모델 포함)
# test_data를 사용하여 unseen data에 대한 성능 확인
leaderboard = predictor.leaderboard(test_data, silent=True)

print("Top 10 Models:")
leaderboard.head(10)

In [ ]:
# 5. 모델 평가
performance = predictor.evaluate(test_data)
print("Test Data Evaluation Results:")
print(performance)

In [ ]:
# 6. 최적의 모델 정보 확인
print("Best Model:", predictor.get_model_best())